In [1]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [33]:
df = pd.read_csv(r'c:\Users\Tim\Documents\CIO\AppCsec6.csv')
df['IDTEXT'] = df.TEXT.apply(lambda s : re.search('\d{1,2}\.\d{1,2}(\.\d{1,2})?', s).group() );
df['TEXT'] = df.TEXT.apply(lambda s : re.sub('^\d{1,2}\.\d{1,2}(\.\d{1,2})?', '', s) );
df[['IDTEXT', 'TEXT']].to_csv(r'c:\Users\Tim\Documents\CIO\AppCsec6_text.csv', index=False)
# print ( df[['IDTEXT', 'TEXT']]  ) 

engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT CONVERT(NVARCHAR(5) , PK_QuestionTypeId) as QTID, code QCODE  FROM fsma_QuestionTypes
""" 
qtypes = pd.read_sql(query,con=conn)  
qtypes.head(2)

In [ ]:
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
""" 
 
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)[:25]}\n" 
s=s+"End Enum\n"
print(s)